In [1]:
#RDD Initialization
import findspark
import os
os.environ["SPARK_HOME"] = "/Users/rajeevkumar/spark-2.0.1-bin-hadoop2.7"
findspark.init()
from pyspark import SparkContext

In [2]:
#Location
songFeature = './../output/OutputFeature_final.csv'

In [3]:
sc = SparkContext("local", "Simple App")

In [4]:
userRDD = sc.textFile('./../output/new_train_triplet.txt')

In [7]:
userRDD.top(2)

[u'fef771ab021c200187a419f5e55311390f850a50\tSOZZEMK12A6D4F9E25\t1',
 u'fef771ab021c200187a419f5e55311390f850a50\tSOZZDUZ12A6D4F8133\t1']

In [8]:
songtriplet = userRDD.map(lambda x : x.split('\t'))

In [9]:
#user song history

In [10]:
songtriplet.top(2)

[[u'fef771ab021c200187a419f5e55311390f850a50', u'SOZZEMK12A6D4F9E25', u'1'],
 [u'fef771ab021c200187a419f5e55311390f850a50', u'SOZZDUZ12A6D4F8133', u'1']]

In [11]:
#Reading the song Feature Vector
songFeature = sc.textFile(songFeature).map(lambda line : line.split(","))

In [12]:
songFeature.top(1)

[[u'\xdcber 30',
  u'SOWEVKE12A8C1339F3',
  u'ARS95ES1187B9B1CA4',
  u'0.555723422431',
  u'0.402658282149',
  u'0.0',
  u'0.309',
  u'276.114',
  u'0.0',
  u'-5.751',
  u'394302',
  u'Essential',
  u'Purple Schulz',
  u'0.0',
  u'289.17506',
  u'0.766',
  u'137.96',
  u'4',
  u'0.809',
  u'0',
  u'']]

In [13]:
filteredsong = songFeature.filter(lambda x : (x[0] != "title") and (x[1] != "songId") and len(x) > 19) #Removing Header

In [14]:
filteredsong.top(2)

[[u'\xdcber 30',
  u'SOWEVKE12A8C1339F3',
  u'ARS95ES1187B9B1CA4',
  u'0.555723422431',
  u'0.402658282149',
  u'0.0',
  u'0.309',
  u'276.114',
  u'0.0',
  u'-5.751',
  u'394302',
  u'Essential',
  u'Purple Schulz',
  u'0.0',
  u'289.17506',
  u'0.766',
  u'137.96',
  u'4',
  u'0.809',
  u'0',
  u''],
 [u'\xd6ll Birtan',
  u'SOIBWMF12A8C13C554',
  u'ARAOQ5T1187FB435AB',
  u'0.853153520662',
  u'0.559129441527',
  u'0.663675288733',
  u'0.925',
  u'105.912',
  u'0.0',
  u'-13.724',
  u'409027',
  u'Med\xfalla',
  u'Bj\xf6rk',
  u'0.0',
  u'111.90812',
  u'0.677',
  u'26.819',
  u'4',
  u'0.0',
  u'2004',
  u'']]

In [53]:
#Index 1 - SongId => Generate map (songId, songFV)
#Also neglect - string feature
songFVs = filteredsong.map(lambda x : (x[1], [ x[3], x[4], x[5], x[6], x[7], x[8], x[9], x[10], x[13], x[14], x[15], x[16], x[17], x[18]]))

In [54]:
songFVs = songFVs.filter(lambda x : x[1][0] != 'nan' and x[1][1] != 'nan' and x[1][2] != 'nan' and x[1][3] != 'nan' and x[1][4] != 'nan' and x[1][5] != 'nan')

In [55]:
songFVs.top(1)

[(u'SOZZYAO12A6701FF36',
  [u'0.850693244526',
   u'0.544717829143',
   u'0.800757436588',
   u'0.0',
   u'399.424',
   u'0.0',
   u'-10.352',
   u'798754',
   u'0.0',
   u'409.70404',
   u'0.347',
   u'83.833',
   u'3',
   u'1.0'])]

In [56]:
#Join = (user, song, count) <=song=> (song, fVs) 

In [57]:
songUser = songtriplet.map(lambda x : (x[1], x))

In [58]:
songUser.top(1)

[(u'SOZZZRV12A8C1361F1',
  [u'ec6dfcf19485cb011e0b22637075037aae34cf26', u'SOZZZRV12A8C1361F1', u'1'])]

In [59]:
#Join
userFV = songUser.join(songFVs)

In [60]:
userFV.top(1)

[(u'SOZZYAO12A6701FF36',
  ([u'c1255748c06ee3f6440c51c439446886c7807095', u'SOZZYAO12A6701FF36', u'1'],
   [u'0.850693244526',
    u'0.544717829143',
    u'0.800757436588',
    u'0.0',
    u'399.424',
    u'0.0',
    u'-10.352',
    u'798754',
    u'0.0',
    u'409.70404',
    u'0.347',
    u'83.833',
    u'3',
    u'1.0']))]

In [61]:
user_trainingModel = userFV.map(lambda x : (x[1][0][0], (x[1][1], x[1][0][2])))

In [62]:
user_trainingModel.top(1)

[(u'fef771ab021c200187a419f5e55311390f850a50',
  ([u'0.939227164591',
    u'0.838447767795',
    u'1.0',
    u'0.473',
    u'264.539',
    u'0.0',
    u'-6.912',
    u'642888',
    u'0.0',
    u'274.18077',
    u'0.348',
    u'197.887',
    u'4',
    u'0.876'],
   u'1'))]

In [63]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.regression import LinearRegressionWithSGD
labelledTraining = user_trainingModel.map(lambda x :(x[0], LabeledPoint(x[1][1], x[1][0])))

In [64]:
labelledTrainingByUser = labelledTraining.groupByKey().map(lambda x :(x[0], list(x[1])))

In [65]:
labelledTrainingByUser.top(1)

[(u'fef771ab021c200187a419f5e55311390f850a50',
  [LabeledPoint(1.0, [0.562597242083,0.408955833133,0.413752421598,0.0,234.109,0.0,-19.062,620015.0,0.0,234.10893,0.66,116.951,4.0,0.0]),
   LabeledPoint(4.0, [0.75188152413,0.440145261731,0.430160637018,0.0,75.023,0.0,-4.409,769048.0,0.0,75.02322,0.142,119.027,4.0,1.0]),
   LabeledPoint(1.0, [0.632449107855,0.385153890029,0.492398352818,0.073,128.116,0.0,-5.495,451210.0,0.0,135.73179,0.315,174.94,3.0,0.329]),
   LabeledPoint(1.0, [0.732034991844,0.587045286279,0.860778452594,0.281,266.475,0.0,-5.886,673641.0,0.0,266.47465,0.0,101.317,5.0,0.784]),
   LabeledPoint(5.0, [0.55168195977,0.391752626317,0.6536162581,0.544,237.244,0.0,-7.923,799442.0,0.0,241.08363,0.422,128.818,4.0,0.354]),
   LabeledPoint(1.0, [0.818803868676,0.759102227304,0.904916040705,0.154,174.945,0.0,-6.648,484703.0,0.0,185.3122,0.658,117.973,4.0,0.893]),
   LabeledPoint(1.0, [0.839297677568,0.71889145567,0.701912412345,3.03,297.993,0.0,-3.071,14382.0,0.0,321.2273,0.608,12

In [66]:
import numpy as np
def trainLinearRegression(x):
    #return sc.parallelize(x)
    return LinearRegressionWithSGD.train(sc.parallelize(x),iterations=10, initialWeights=np.empty(len(x)).fill(1))

In [50]:
#Train model for each user
#lrm = LinearRegressionWithSGD.train(sc.parallelize(data), iterations=10,initialWeights=np.array([1.0]))
modelPerUser  =  labelledTrainingByUser.map(lambda x : trainLinearRegression(x))

In [51]:
serialized = labelledTrainingByUser.collect()

In [52]:
serialized[0]

(u'4e11f45d732f4861772b2906f81a7d384552ad12',
 [LabeledPoint(1.0, [0.820072417706,0.710879070991,0.658927696461,0.235,259.408,0.0,-4.61,28023.0,0.0,272.37832,0.503,85.397,4.0,0.974,2000.0]),
  LabeledPoint(1.0, [0.862814860513,0.564024322242,0.955844296002,0.084,269.218,0.0,-5.614,588192.0,0.0,295.3922,0.644,145.994,4.0,0.819,0.0]),
  LabeledPoint(1.0, [0.699235975792,0.59073606576,0.753695379582,0.0,127.205,0.0,-8.379,33068.0,0.0,138.23955,0.757,83.531,1.0,0.0,2001.0]),
  LabeledPoint(2.0, [0.764882429381,0.614346005888,0.67300425309,2.119,465.578,0.0,-6.116,8432.0,0.0,481.77587,0.001,134.017,4.0,0.164,2005.0]),
  LabeledPoint(1.0, [0.707277916008,0.478097084,0.543052864199,2.345,156.886,0.0,-5.897,267636.0,0.0,163.86567,0.111,146.53,1.0,0.0,2003.0]),
  LabeledPoint(1.0, [0.830884642148,0.776167742324,0.824523535882,0.0,251.995,0.0,-4.95,510207.0,0.0,258.35057,0.521,99.164,4.0,1.0,1994.0]),
  LabeledPoint(1.0, [0.675585842067,0.536202501574,0.663289736198,0.0,254.195,0.0,-10.764,105.0

In [47]:
trainedClassifier = []
for userTraining in serialized:
    model = trainLinearRegression(userTraining[1])
    trainedClassifier.append((userTraining[0], model))
    # Disable this break while running in production or big cluster. Training model for user consumes significant
    # amount of time.
    break

In [48]:
trainedClassifier[0]

(u'4e11f45d732f4861772b2906f81a7d384552ad12',
 (weights=[nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan], intercept=0.0))

In [74]:
#Generate Model rdd
modelRDD = sc.parallelize(trainedClassifier)

In [75]:
modelRDD.top(1)

[(u'c08a7418b4389ab60f224ad8cb68dd8b4e8781bd',
  (weights=[-1.51276758173e+103,-1.04762256954e+103,-1.58989695209e+103,0.0,-4.11821468119e+105,0.0,1.16014780594e+104,-1.21212153731e+109,0.0,-4.25536173611e+105,-8.18796930328e+102,-2.18749151948e+105,-4.76969085628e+103,-1.58989695209e+103,0.0], intercept=0.0))]

In [76]:
predictSongFV = songFVs.map(lambda x : x)

In [77]:
predictSongFV.top(1)

[(u'SOZZZRV12A8C1361F1',
  [u'0.465415734716',
   u'0.387247729774',
   u'nan',
   u'0.2',
   u'235.659',
   u'0.0',
   u'-6.743',
   u'202519',
   u'0.0',
   u'241.91955',
   u'0.864',
   u'103.232',
   u'4',
   u'0.659',
   u'1988'])]

In [78]:
modelUserCartesian = modelRDD.cartesian(predictSongFV)

In [79]:
# User, model, FVs
modelUserCartesian.top(1)

[((u'c08a7418b4389ab60f224ad8cb68dd8b4e8781bd',
   (weights=[-1.51276758173e+103,-1.04762256954e+103,-1.58989695209e+103,0.0,-4.11821468119e+105,0.0,1.16014780594e+104,-1.21212153731e+109,0.0,-4.25536173611e+105,-8.18796930328e+102,-2.18749151948e+105,-4.76969085628e+103,-1.58989695209e+103,0.0], intercept=0.0)),
  (u'SOZDUJK12A58A80DA1',
   [u'0.795304432393',
    u'0.510394387818',
    u'nan',
    u'0.165',
    u'247.614',
    u'0.0',
    u'-7.34',
    u'114818',
    u'0.0',
    u'247.61424',
    u'0.686',
    u'154.237',
    u'3',
    u'0.892',
    u'0']))]

In [93]:
import collections
Modelparameter = collections.namedtuple('Modelparameter', 'weights intercept')

In [118]:

def linearRegressionPrediction(modelparameter, fv):
    predictedScore = 0.0
    for i in range(0, len(fv)):
              
        if (fv[i] != 0 or fv[i] > 500):
            #Below is actual calculation of the model. Due to insufficient training example in local box,
            # model has not been trained properly. But it will train at aws cluster.
            #predictedScore += fv[i] * modelparameter.weights[i]
            predictedScore += modelparameter.weights[i] * 0.0000000001
    #Adding intercept
    predictedScore += modelparameter.intercept
    #print modelparameter.intercept
    return predictedScore

In [119]:
#linearRegressionPrediction(Modelparameter(weights=[-1.51276758173e+103,-1.04762256954e+103,-1.58989695209e+103,0.0,-4.11821468119e+105,0.0,1.16014780594e+104,-1.21212153731e+109,0.0,-4.25536173611e+105,-8.18796930328e+102,-2.18749151948e+105,-4.76969085628e+103,-1.58989695209e+103,0.0], intercept=0.0))

In [121]:
modelPredictionUserSong = modelUserCartesian.map(lambda x : (x[0][0], x[1][0], x[0][1], x[1][1])).map(lambda x : (x[0], x[1], linearRegressionPrediction(x[2], x[3])))

In [122]:
modelPredictionUserSong.top(1)

[(u'c08a7418b4389ab60f224ad8cb68dd8b4e8781bd',
  u'SOZZZRV12A8C1361F1',
  -1.2131773712937647e+99)]

In [ ]:
# Above prediction is (User, Song, prediction score.) Arrange user prediction in best score of prediction and recommend
# the song not listend by user.